In [72]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

file_path = 'data/raw_data.csv'  
df = pd.read_csv(file_path)
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
print(df.head())

/var/folders/xg/6886p3rj5kb44k2smttjctf80000gn/T/ipykernel_56459/2703093388.py:7: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Shape: (2260701, 151)
Columns: ['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_s

In [73]:
print(df.isnull().sum().sort_values(ascending=False))

# data cleaning
# drop duplicates
df = df.drop_duplicates()

#delete columns with more than 50% missing values
threshold = len(df) * 0.5
df_cleaned = df.dropna(thresh=threshold, axis=1)

print("Columns after dropping those with >50% missing values:", df_cleaned.shape)


member_id                                     2260701
orig_projected_additional_accrued_interest    2252050
hardship_end_date                             2249784
hardship_start_date                           2249784
hardship_type                                 2249784
                                               ...   
policy_code                                        33
revol_bal                                          33
fico_range_high                                    33
fico_range_low                                     33
id                                                  0
Length: 151, dtype: int64
Columns after dropping those with >50% missing values: (2260701, 107)


In [74]:
# drop unnecessary columns
columns_to_drop = ['id', 'url', 'title', 'zip_code', 'policy_code', 'application_type', 'hardship_flag', 'debt_settlement_flag']
columns_to_drop.extend(["desc", "mths_since_last_delinq", "mths_since_last_record", "next_pymnt_d", "tot_hi_cred_lim"])
columns_to_drop.extend(["mths_since_last_major_derog", "total_bal_ex_mort", "total_bc_limit", "total_il_high_credit_limit"])
columns_to_drop.extend(["member_id", "emp_title", "tax_liens"])
existing_to_drop = [col for col in columns_to_drop if col in df_cleaned.columns]
print("将被删除的列：", existing_to_drop)

df_cleaned = df_cleaned.drop(columns=existing_to_drop)
print("shape", df_cleaned.shape)


将被删除的列： ['id', 'url', 'title', 'zip_code', 'policy_code', 'application_type', 'hardship_flag', 'debt_settlement_flag', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'emp_title', 'tax_liens']
shape (2260701, 93)


In [75]:
# deal with date columns
date_columns = ['issue_d', 'last_pymnt_d', 'last_credit_pull_d']
for col in date_columns:
    if col in df_cleaned.columns:
        df_cleaned[col] = pd.to_datetime(df_cleaned[col], errors='coerce')
        df_cleaned[col+'_year'] = df_cleaned[col].dt.year
        df_cleaned[col+'_month'] = df_cleaned[col].dt.month
        df_cleaned = df_cleaned.drop(columns=[col])
print("Date columns processed. Remaining columns:", df_cleaned.columns.tolist())
print("Shape after date processing:", df_cleaned.shape)

/var/folders/xg/6886p3rj5kb44k2smttjctf80000gn/T/ipykernel_56459/2374708608.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cleaned[col] = pd.to_datetime(df_cleaned[col], errors='coerce')
/var/folders/xg/6886p3rj5kb44k2smttjctf80000gn/T/ipykernel_56459/2374708608.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cleaned[col] = pd.to_datetime(df_cleaned[col], errors='coerce')
/var/folders/xg/6886p3rj5kb44k2smttjctf80000gn/T/ipykernel_56459/2374708608.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cleaned[col] = pd.to_datetime(df_cleaned[col], errors='coer

Date columns processed. Remaining columns: ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'loan_status', 'pymnt_plan', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_pa

In [76]:
print('Number of rows having all values as null:')
print(df_cleaned.isnull().all(axis=1).sum())

print('Number of columns having all values as null:')
print(df_cleaned.isnull().all(axis=0).sum())

Number of rows having all values as null:
33
Number of columns having all values as null:
0


In [77]:
#  default based on 'loan_status'
# “Charged Off”、“Default”、“Late” = 1
def create_target(status):
    if status in ['Charged Off', 'Default', 'Late (31-120 days)', 'Does not meet the credit policy. Status:Charged Off']:
        return 1
    elif status in ['Fully Paid', 'Does not meet the credit policy. Status:Fully Paid']:
        return 0
    else:
        return np.nan  


df_cleaned['default'] = df_cleaned['loan_status'].apply(create_target)
df_cleaned = df_cleaned.dropna(subset=['default'])




In [78]:
#drop rows with all null values
df_cleaned = df_cleaned.dropna(how='all')
print("Shape after dropping rows with all null values:", df_cleaned.shape)

Shape after dropping rows with all null values: (1369566, 97)


In [79]:
categorical_cols = df_cleaned.select_dtypes(include=['object']).columns.tolist()
print("Categorical columns:", categorical_cols)

Categorical columns: ['term', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'verification_status', 'loan_status', 'pymnt_plan', 'purpose', 'addr_state', 'earliest_cr_line', 'initial_list_status', 'disbursement_method']


In [80]:
# term: "36 months" -> 36，"60 months" -> 60
df_cleaned['term'] = df_cleaned['term'].str.extract('(\d+)').astype(float)

# emp_length: （< 1 year -> 0, 10+ years -> 10）
def convert_emp_length(x):
    if pd.isnull(x):
        return np.nan
    if '<' in x:
        return 0
    if '10+' in x:
        return 10
    return int(x.strip().split()[0])

df_cleaned['emp_length'] = df_cleaned['emp_length'].apply(convert_emp_length)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
/var/folders/xg/6886p3rj5kb44k2smttjctf80000gn/T/ipykernel_56459/3441514332.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_cleaned['term'] = df_cleaned['term'].str.extract('(\d+)').astype(float)


In [82]:
# label encoding for categorical columns

from sklearn.preprocessing import OneHotEncoder
OneHot_encoder = OneHotEncoder()
for col in categorical_cols:
    if col in df_cleaned.columns:
        df_cleaned[col] = df_cleaned[col].astype(str)  
        df_cleaned[col] = OneHot_encoder.fit_transform(df_cleaned[[col]]).toarray()



In [83]:
# deal with missing values ,# using median for numerical columns
from sklearn.impute import SimpleImputer
num_cols = df_cleaned.select_dtypes(include=[np.number]).columns.tolist()
imputer = SimpleImputer(strategy='median')
df_cleaned[num_cols] = imputer.fit_transform(df_cleaned[num_cols])

In [84]:
print("Shape of the data frame :",df_cleaned.shape)
print("===================================================")
print("Information about the data frame :",df_cleaned.info())
print("===================================================")
print("The data types of columns of data frame",df_cleaned.dtypes)

Shape of the data frame : (1369566, 97)
<class 'pandas.core.frame.DataFrame'>
Index: 1369566 entries, 0 to 2260697
Data columns (total 97 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   loan_amnt                   1369566 non-null  float64
 1   funded_amnt                 1369566 non-null  float64
 2   funded_amnt_inv             1369566 non-null  float64
 3   term                        1369566 non-null  float64
 4   int_rate                    1369566 non-null  float64
 5   installment                 1369566 non-null  float64
 6   grade                       1369566 non-null  float64
 7   sub_grade                   1369566 non-null  float64
 8   emp_length                  1369566 non-null  float64
 9   home_ownership              1369566 non-null  float64
 10  annual_inc                  1369566 non-null  float64
 11  verification_status         1369566 non-null  float64
 12  loan_status          

In [85]:
df_cleaned['loan_status'].value_counts()



loan_status
0.0    1101007
1.0     268559
Name: count, dtype: int64

In [86]:
print("Shape:", df_cleaned.shape)
# 可保存处理后的数据
df_cleaned.to_csv('data/lendingclub_cleaned.csv', index=False)

Shape: (1369566, 97)
